In [1]:
import pandas as pd

def tsv_to_dataframe(file_path):
    df = pd.read_csv(file_path, sep='\t', skiprows=[0, 2])
    return df

# 使用方法
# dataframe = tsv_to_dataframe('your_file_path.tsv')


In [2]:
df = tsv_to_dataframe('primary.tsv')
df = df[['common formula of materials','chemical formula', 'Tc (of this sample) recommended']]
df = df.dropna(subset=['chemical formula','Tc (of this sample) recommended'])

In [3]:
import re

def modify_string(s):
    # 匹配以数字结尾的字符串
    if re.match(r'.*\d$', s):
        return s
    # 匹配以"1-x"或"3+y"这样的形式结尾的字符串
    elif re.match(r'.*\d[-+][a-zA-Z]$', s) or re.match(r'.*\d\+-[a-zA-Z]$', s):
        return re.sub(r'(\d)[-+][a-zA-Z]$', r'\1', s)
    # 匹配以字母'x'、'y'、'z'（不区分大小写）结尾的字符串
    elif re.match(r'.*[xXyYzZ]$', s):
        return s[:-1] + '?'
    else:
        return s
df.iloc[:, 1] = df.iloc[:, 1].apply(modify_string)

In [4]:
import pandas as pd
import numpy as np
import re

# 假设df是你的DataFrame

# 找出第二列包含"?"的行
mask = df.iloc[:, 1].str.contains("\?", na=False)

# 收集要删除的行的索引
drop_index = []

# 对于这些行，检查第一列的元素
for idx in df[mask].index:
    # 如果第一列元素为空，则记录索引以便后续删除
    if pd.isnull(df.iloc[idx, 0]):
        drop_index.append(idx)
    else:
        # 如果第一列元素的末尾不是数，则记录索引以便后续删除
        if not re.match(r'.*\d$', df.iloc[idx, 0]):
            drop_index.append(idx)
        else:
            # 如果第一列元素的末尾是数，则用这个数替换第二列元素的"?"
            last_num = re.findall(r'\d+\.?\d*$', df.iloc[idx, 0])
            if last_num:
                df.iloc[idx, 1] = df.iloc[idx, 1].replace("?", last_num[0])

# 删除记录的行
df.drop(drop_index, inplace=True)

# 重置索引
df.reset_index(drop=True, inplace=True)


In [5]:
df

,common formula of materials,chemical formula,Tc (of this sample) recommended
0,"(Ba,La)2CuO4",Ba0.2La1.8Cu1O4,29.00
1,"(Ba,La,Ag)2CuO4",Ba0.1La1.9Ag0.1Cu0.9O4,26.00
2,"(Ba,La,)2CuO4",Ba0.1La1.9Cu1O4,19.00
3,"(Ba,La)2CuO4",Ba0.15La1.85Cu1O4,22.00
4,"(Ba,La)2CuO4",Ba0.3La1.7Cu1O4,23.00
...,...,...,...
25044,Hf0.7Lu0.5P1.45S0.55,Hf0.5Lu0.5P1.45S0.55,5.87
25045,Hf0.8Lu0.6P1.45S0.55,Hf0.4Lu0.6P1.45S0.55,5.00
25046,Hf0.6Y0.3P1.55S0.45,Hf0.7Y0.3P1.45S0.55,4.53
25047,Hf0.5Y0.4P1.55S0.45,Hf0.6Y0.4P1.45S0.55,4.74


In [8]:
# 假设你的DataFrame叫做clean_df
df2 = df[['chemical formula', 'Tc (of this sample) recommended']]
new_headers = ['Materials_Name', 'Tc']
df2.columns = new_headers
df2.to_csv('./output2.csv', index=False)

In [7]:
df[df.iloc[:, 1].str.contains("\?")]

,common formula of materials,chemical formula,Tc (of this sample) recommended
